In [124]:
from importlib import reload
reload(aux)
reload(con)
reload(dt)

<module 'src.daily_data' from 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\src\\daily_data.py'>

In [125]:
import pandas as pd
import src.daily_data as dt
import src.auxiliary_functions as aux
import src.condition as con
from datetime import time, datetime
import src.info as info
import src.in_out as io
from src.density import Density

In [116]:
path_list = io.get_path('C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\')

In [3]:
infolist = [info.get_info(path) for path in path_list]

In [4]:
paired_days = dt.to_match_days(infolist)

In [5]:
pair = paired_days[0]

In [6]:
mid_price1_all = info.__calc_mp(pair[0].hourly_data)

In [7]:
mid_price2_all = info.__calc_mp(pair[1].hourly_data)

In [8]:
mid_price1 = mid_price1_all[18].reset_index(drop=True)

In [9]:
mid_price2 = mid_price2_all[18].reset_index(drop=True)

In [126]:
spred = aux.find_spread((mid_price1.mid_price,mid_price2.mid_price),a_PNLTICK=1.4,a_TICKSIZE=0.0201,b_PNLTICK=6.25,b_TICKSIZE=0.0001)

haata


In [127]:
change = aux.find_change(spred)

In [128]:
duration = aux.find_duramp(change)

In [129]:
df = pd.DataFrame([change,duration.duration,duration.amplitude])

In [130]:
df = df.transpose()

In [131]:
directory_path = 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\out\\'

In [132]:
writer = pd.ExcelWriter(directory_path+ "df"+'.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1')
writer.save()

In [ ]:
daily_datas = []

In [ ]:
for pair in paired_days:
    product = pair[0].product + "_" + pair[1].product
    paired_hours=dt.to_match_hour(pair[0],pair[1])
    gunluk_veri = dt.DailyData(mid_price_list=paired_hours,date=pair[0].date, name= product)
    gunluk_veri.spread_hesapla()
    daily_datas.append(gunluk_veri)

In [ ]:
products={}

In [ ]:
for gunluk in daily_datas:
    if gunluk.name in products:
        products[gunluk.name].append(gunluk)
    else:
        products[gunluk.name] = [gunluk]

In [ ]:
combined_days = {}
combined_days_detail = {}
combined_days_con = {}
combined_days_con_detail = {}

In [ ]:
stop=False
while not stop:
    try:
        urun = products.popitem()
    except:
        stop=True
    df = dt.get_all_data(urun[1])
    con_df = con.scan(df)
    combined_days[urun[0]] = df
    combined_days_con[urun[0]] = con_df
    combined_days_detail[urun[0]] = aux.get_detail(df)
    combined_days_con_detail[urun[0]] = aux.get_detail(con_df)

In [ ]:
all_products = list(combined_days.keys())

In [ ]:
hour_series = pd.date_range('2018-01-01-18', periods=23, freq='H').time

In [ ]:
all_density= {}

In [ ]:
for product in all_products:
    densities = {}
    for hour in hour_series:
        df = combined_days[product].loc[hour]
        densities[hour]= Density(df)
    all_density[product] = densities

# Example

In [ ]:
data_of_the_eighteenth_clock = all_density['6BU8_6CU8'][time(18)]

data_of_the_eighteenth_clock.data

data_of_the_eighteenth_clock.density_data

In [ ]:
data_of_the_eighteenth_clock.draw()